In [1]:
import sys; sys.path.insert(0, "..")
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./data/train_A.tsv', delimiter="\t")
df.head()

,user_id,product_id,event_type,ad,time_stamp
0,0000000_A,00009250_a,1,-1,2017-04-08 12:09:04.629
1,0000000_A,00009250_a,1,-1,2017-04-27 12:55:57.783
2,0000000_A,00014068_a,1,-1,2017-04-08 11:57:53.746
3,0000000_A,00001254_a,1,-1,2017-04-08 12:04:26.008
4,0000000_A,00003316_a,1,-1,2017-04-08 12:05:31.326


In [3]:
df = df.sort_values(by=["user_id", "product_id"])
df = df[df["ad"]!=0]
df["event_type"] = df["event_type"].astype(float)

In [4]:
action_count = df.groupby(["user_id", "product_id"]).size().values
action_count

array([1, 1, 1, ..., 2, 5, 3])

In [5]:
# タイムスタンプは一番最新
action_df = df.sort_values(by=["user_id", "product_id", "time_stamp"],ascending=[True, True, False]).drop_duplicates(["user_id", "product_id"])
action_df["action_count"] = action_count
action_df

,user_id,product_id,event_type,ad,time_stamp,action_count
3,0000000_A,00001254_a,1.0,-1,2017-04-08 12:04:26.008,1
5,0000000_A,00003009_a,1.0,-1,2017-04-14 13:55:07.446,1
4,0000000_A,00003316_a,1.0,-1,2017-04-08 12:05:31.326,1
12,0000000_A,00003524_a,1.0,-1,2017-04-27 13:26:46.064,1
6,0000000_A,00004433_a,1.0,-1,2017-04-27 13:30:18.503,2
...,...,...,...,...,...,...
2524516,0058657_A,00009935_a,1.0,-1,2017-04-24 05:18:23.219,1
2524510,0058657_A,00011943_a,3.0,1,2017-04-21 08:19:00.468,10
2524522,0058657_A,00013041_a,1.0,-1,2017-04-07 01:31:10.841,2
2524533,0058657_A,00013336_a,0.0,-1,2017-04-24 06:36:51.952,5


In [6]:
# ユーザ数を100人に絞る(ぼくのmacのスペック、、)
num_users = 10
valid_user_ids = sorted(action_df.user_id.unique())[:num_users]
action_df = action_df[action_df.user_id <= max(valid_user_ids)]

In [7]:
# 学習用とテスト用にデータを分割
# 各ユーザの直近5件の映画を評価用に使い、それ以外を学習用にする
action_df["time_stamp"] = pd.to_datetime(action_df["time_stamp"])
action_df["rating_order"] = action_df.groupby("user_id")["time_stamp"].rank(ascending=False, method="first")
train_df = action_df[action_df["rating_order"] > 5]
test_df = action_df[action_df["rating_order"] <= 5]

/root/.cache/pypoetry/virtualenvs/recommendersystems-9TtSrW0h-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/root/.cache/pypoetry/virtualenvs/recommendersystems-9TtSrW0h-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [8]:
minmum_num_rating = 0
filtered_train_df = train_df.groupby("product_id").filter(lambda x: len(x["product_id"]) >= minmum_num_rating)

In [9]:
# 行列分解用に行列を作成
train_high_event = filtered_train_df[train_df["event_type"] >= 2]
train_high_event.head()

,user_id,product_id,event_type,ad,time_stamp,action_count,rating_order
876635,0000003_A,00000498_a,3.0,1,2017-04-04 01:13:00.964,4,16.0
876628,0000003_A,00003603_a,3.0,1,2017-04-04 10:20:18.549,5,14.0
876619,0000003_A,00012529_a,3.0,1,2017-04-01 03:03:28.189,6,23.0
952482,0000005_A,00002460_a,3.0,1,2017-04-24 07:05:52.974,16,8.0
952438,0000005_A,00008894_a,3.0,1,2017-04-15 08:45:00.803,12,30.0


In [10]:
unique_user_ids = sorted(train_high_event.user_id.unique())
unique_product_ids = sorted(train_high_event.product_id.unique())
user_id2index = dict(zip(unique_user_ids, range(len(unique_user_ids))))
product_id2index = dict(zip(unique_product_ids, range(len(unique_product_ids))))

In [11]:
from scipy.sparse import lil_matrix
alpha = 0

matrix = lil_matrix((len(unique_product_ids), len(unique_user_ids)))
for i, row in train_high_event.iterrows():
    user_index = user_id2index[row["user_id"]]
    product_index = product_id2index[row["product_id"]]
    matrix[product_index, user_index] = row["event_type"]*alpha
matrix

<7x2 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in List of Lists format>

In [12]:
# 行列分解用に行列を
train_high_event = filtered_train_df[train_df["event_type"] >= 2]
        
unique_user_ids = sorted(train_high_event.user_id.unique())
unique_product_ids = sorted(train_high_event.product_id.unique())
user_id2index = dict(zip(unique_user_ids, range(len(unique_user_ids))))
product_id2index = dict(zip(unique_product_ids, range(len(unique_product_ids))))

In [14]:
import implicit
# 因子数
factors = 10
# エポック数
n_epochs = 50

# モデルの初期化
model = implicit.als.AlternatingLeastSquares(
    factors=factors, iterations=n_epochs, calculate_training_loss=True, random_state=1
)

In [15]:
# 学習
model.fit(matrix)
print(matrix)

  0%|          | 0/50 [00:00<?, ?it/s]